In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from pprint import pprint
import re
import emoji
import os
import sys
import string

from sklearn.model_selection import train_test_split
from transformers import RobertaTokenizer, TFRobertaForSequenceClassification
from datasets import Dataset



/home/ubuntu/dl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-10-02 07:40:40.295073: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-02 07:40:40.311878: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-02 07:40:40.316879: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-02 07:40:40.330257: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimize

In [2]:

import tensorflow as tf
print(tf.reduce_sum(tf.random.normal([1000, 1000])))
print(tf.config.list_physical_devices('GPU'))

tf.Tensor(1380.561, shape=(), dtype=float32)
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


I0000 00:00:1727854878.420408    2580 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1727854878.466467    2580 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1727854878.466752    2580 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1727854878.470217    2580 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [3]:
df = pd.read_csv('data.csv')
pprint(df.head(20))

    label                                              tweet
0       0   @user when a father is dysfunctional and is s...
1       0  @user @user thanks for #lyft credit i can't us...
2       0                                bihday your majesty
3       0  #model   i love u take with u all the time in ...
4       0             factsguide: society now    #motivation
5       0  [2/2] huge fan fare and big talking before the...
6       0   @user camping tomorrow @user @user @user @use...
7       0  the next school year is the year for exams.ð...
8       0  we won!!! love the land!!! #allin #cavs #champ...
9       0   @user @user welcome here !  i'm   it's so #gr...
10      0   â #ireland consumer price index (mom) climb...
11      0  we are so selfish. #orlando #standwithorlando ...
12      0  i get to see my daddy today!!   #80days #getti...
13      1  @user #cnn calls #michigan middle school 'buil...
14      1  no comment!  in #australia   #opkillingbay #se...
15      0  ouch...junior

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56745 entries, 0 to 56744
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   56745 non-null  int64 
 1   tweet   56745 non-null  object
dtypes: int64(1), object(1)
memory usage: 886.8+ KB


In [5]:
for i in range(20):
    print(df['label'][i]," : ",df['tweet'][i])

0  :   @user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction.   #run
0  :  @user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx.    #disapointed #getthanked
0  :    bihday your majesty
0  :  #model   i love u take with u all the time in urð±!!! ððððð¦ð¦ð¦  
0  :   factsguide: society now    #motivation
0  :  [2/2] huge fan fare and big talking before they leave. chaos and pay disputes when they get there. #allshowandnogo  
0  :   @user camping tomorrow @user @user @user @user @user @user @user dannyâ¦
0  :  the next school year is the year for exams.ð¯ can't think about that ð­ #school #exams   #hate #imagine #actorslife #revolutionschool #girl
0  :  we won!!! love the land!!! #allin #cavs #champions #cleveland #clevelandcavaliers  â¦ 
0  :   @user @user welcome here !  i'm   it's so #gr8 ! 
0  :   â #ireland consumer price index (mom) climbed from previous 0.2% to 0.5% in may  

### Preprocessing

In [6]:
def clean_text(text):
    text = text.lower()

    #demojize the text
    text = emoji.demojize(text)

    # Remove user mentions (@username)
    text = re.sub(r'@\w+', '', text)

    # Remove URLs
    text = re.sub(r'https?://\S+|www\.\S+', '', text)

    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)

    # Remove special characters, punctuation, and numbers
    text = re.sub(r'[^\w\s,\.!?]', '', text)
    # Remove hashtags but keep the words
    text = re.sub(r'#', '', text)

    # Remove emojis or replace them with a placeholder (optional)
    text = re.sub(r'[^\x00-\x7F]+', '', text)  # Keep ASCII characters only

    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text

df['cleaned_tweet'] = df['tweet'].apply(clean_text)


In [7]:
# Take a random sample of 50 rows from the DataFrame
sampled_df = df.sample(n=100, random_state=42)

# Print the label, cleaned_tweet, and tweet for each sample
for i in range(100):
    print(sampled_df['label'].iloc[i], "   :   ", sampled_df['cleaned_tweet'].iloc[i], "              :--------------------  ", sampled_df['tweet'].iloc[i])

1    :    kim k is alright but honestly so many niggas been in that id rather not.. and miley got unfollowed that bitch is just strange.               :--------------------   Kim K is alright but honestly so many niggas been in that I'd rather not.. And Miley got unfollowed that bitch is just strange.
1    :    rt the stars wink knowingly and smile at each other when they see two lovers kissing on a secluded bridge on a balmy autumn nig8230               :--------------------   RT @BigJonBul: The stars wink knowingly and smile at each other when they see two lovers kissing on a secluded bridge on a balmy autumn nig&#8230;
1    :    rt 8220 8220 my sister shittin on you hos once again wodeh bout dat action               :--------------------   RT @Sassymermaidxo: &#8220;@VSamone: &#8220;@ZvckSlvt3r704: My sister shittin on you hos once again "@VSamone: wodeh bout dat action http://t.co/mDfLBr&#8230;
1    :    those tell me about yourself lookin ass niggas ... ima crazy bitch thats all y

In [8]:
texts = df['cleaned_tweet'].tolist()
labels = df['label'].tolist()
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

#### Tokenization

In [9]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')


/home/ubuntu/dl/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [10]:
# Tokenize the texts

AUTOTUNE = tf.data.experimental.AUTOTUNE

def tokenize_data(texts, labels):
    encodings = tokenizer(texts, truncation=True, padding=True, max_length=128, return_tensors='tf')
    return encodings['input_ids'], encodings['attention_mask'], np.array(labels)

train_input_ids, train_attention_masks, train_labels = tokenize_data(train_texts, train_labels)
val_input_ids, val_attention_masks, val_labels = tokenize_data(val_texts, val_labels)

#### TF Dataset Creation

In [11]:
# Create TensorFlow Datasets
train_dataset = tf.data.Dataset.from_tensor_slices(({'input_ids': train_input_ids, 'attention_mask': train_attention_masks}, train_labels))
val_dataset = tf.data.Dataset.from_tensor_slices(({'input_ids': val_input_ids, 'attention_mask': val_attention_masks}, val_labels))

#### Batch Creation

In [12]:
# Define the data pipeline (no need to map tokenize_data again here)
train_data = (train_dataset
              .shuffle(buffer_size=1000)  # Shuffle the dataset with buffer size
              .batch(16)  # Batch size for training
              .prefetch(buffer_size=AUTOTUNE))  # Prefetch for efficient training

val_data = (val_dataset
            .batch(64)  # Batch size for validation (larger as it's faster for eval)
            .prefetch(buffer_size=AUTOTUNE))


#### Modelling

In [13]:
model = TFRobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

In [14]:
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5, epsilon=1e-08)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[metric])


In [15]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir='./logs')
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath='./best_model', save_best_only=True, monitor='val_accuracy', mode='max')

In [ ]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=3,
    callbacks=[tensorboard_callback, checkpoint_callback]
)


Epoch 1/3


I0000 00:00:1727855015.363794    2729 service.cc:146] XLA service 0x7bd65db6fc00 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1727855015.363829    2729 service.cc:154]   StreamExecutor device (0): NVIDIA RTX A6000, Compute Capability 8.6
2024-10-02 07:43:35.370056: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-10-02 07:43:35.385936: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907
I0000 00:00:1727855015.468972    2729 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1843/2838 [==================>...........] - ETA: 1:40 - loss: 0.2295 - accuracy: 0.9093

In [ ]:
eval_results = model.evaluate(val_dataset)
print(f"Evaluation results: {eval_results}")


In [ ]:
model.save_pretrained('./final_model')
